# Loading data into Postgresql databases on local server.

We are going to load the tables we have normalized and validated earlier into Postgresql database local server.  
First, we have to establish a connection to the server using library psycopg2. I have created the login parameters in a separate file. I have also created a configuration file to read the defined parameters into a dictionary.

## Establishing connection to Postgresql server
I already created the database that I am going to use with the name Human_resource. So, I am going to directly connect to that database

In [91]:
import psycopg2
from config import config

# Connect to the PostgreSQL database server
conn = None
try:
    # read connection parameters
    params = config()

    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(**params)
    if conn:
        print('Connected to database')

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

cursor = conn.cursor()

Connecting to the PostgreSQL database...
Connected to database


## Creating tables in database
Before we load the data, we have to create tables to contain them.

In [93]:
# Postgres command to create the tables
create_tables = (        
        """
        CREATE TABLE IF NOT EXISTS public.emp_info
        (
            "EmployeeID" integer NOT NULL,
            "StreetAddress" character varying(75),
            "City" character varying(50),
            "ZipCode" integer,
            "State_code" character varying(2) NOT NULL,
            "Country_code" character varying(5) NOT NULL,
            "Start_Date" date,
            "Termination_Date" date,
            "Active_Status" smallint,
            start_year integer,
            termination_year integer,
            tenure_months numeric(5, 2),
            tenure_years numeric(5, 2),
            PRIMARY KEY ("EmployeeID")
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS public.emp_demo
        (
            "EmployeeID" integer NOT NULL,
            "Gender" character varying(6),
            "Gender_Identity" character varying(12),
            "Race_Ethnicity" character varying(30),
            "Veteran" smallint,
            "Disability" smallint,
            "Education" character varying(50),
            "Sexual_Orientation" character varying(30),
            "Age" smallint,
            "DOB" date,
            PRIMARY KEY ("EmployeeID")
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS public.job_details
        (
            "Job_Profile" integer NOT NULL,
            "Department" character varying(20),
            "Job_title" character varying(35),
            "Salary" numeric(7, 2),
            level character varying(50),
            "Bonus_pct" numeric(3, 2),
            PRIMARY KEY ("Job_Profile")
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS public.office
        (
            office_id integer NOT NULL,
            office_name character varying(20),
            office_type character varying(30),
            PRIMARY KEY (office_id)
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS public.state
        (
            "State_code" character varying(2) NOT NULL,
            "State_name" character varying(50),
            PRIMARY KEY ("State_code")
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS public.country
        (
            "Country_code" character varying(5) NOT NULL,
            "Country_name" character varying(50),
            PRIMARY KEY ("Country_code")
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS public.main_details
        (
            "EmployeeID" integer NOT NULL,
            "First_name" character varying(50),
            "Surname" character varying(50),
            "Job_Profile" integer NOT NULL,
            office_id integer NOT NULL,
            PRIMARY KEY ("EmployeeID")
        )
        """
)

In [94]:
# Command to add foreign keys
add_fks = (
        """
        ALTER TABLE IF EXISTS public.emp_info
        ADD FOREIGN KEY ("Country_code")
        REFERENCES public.country ("Country_code") MATCH SIMPLE
        ON UPDATE CASCADE
        ON DELETE CASCADE
        """,
        """
        ALTER TABLE IF EXISTS public.emp_info
        ADD FOREIGN KEY ("State_code")
        REFERENCES public.state ("State_code") MATCH SIMPLE
        ON UPDATE CASCADE
        ON DELETE CASCADE
        """,
        """
        ALTER TABLE IF EXISTS public.main_details
        ADD FOREIGN KEY ("EmployeeID")
        REFERENCES public.emp_info ("EmployeeID") MATCH SIMPLE
        ON UPDATE CASCADE
        ON DELETE CASCADE
        """,
        """
        ALTER TABLE IF EXISTS public.main_details
        ADD FOREIGN KEY ("Job_Profile")
        REFERENCES public.job_details ("Job_Profile") MATCH SIMPLE
        ON UPDATE CASCADE
        ON DELETE CASCADE
        """,
        """
        ALTER TABLE IF EXISTS public.main_details
        ADD FOREIGN KEY (office_id)
        REFERENCES public.office (office_id) MATCH SIMPLE
        ON UPDATE CASCADE
        ON DELETE CASCADE
        """,
        """
        ALTER TABLE IF EXISTS public.main_details
        ADD FOREIGN KEY ("EmployeeID")
        REFERENCES public.emp_demo ("EmployeeID") MATCH SIMPLE
        ON UPDATE CASCADE
        ON DELETE CASCADE
        """
)

In [95]:
# execute creation commands
for command in create_tables:
    cursor.execute(command)

In [96]:
# execute alteration commands
for command in add_fks:
    cursor.execute(command)

In [97]:
# commit creation of tables
conn.commit()